# clustering notes and documents based on tags 
for a while i have been using markdown to take notes and tags are good way to sort and find related documents.
clustering notes based on tags could be usefull way to  find and review related pieces of information in your library of notes.

## vectorisation and embeding
to efectively cluster documentts based on tags we could go with few aproach. 
- create giant booolean vector where 1 means that document has that tag, and 0 means it doesnt.

this aproach can work but it doesnt take into acount that some tags are related. for example document with the tags:[#math] and [#python] would be as far apart(norm/distance) as [#cpp] and [#foodrecipes]. math and python are clearly more related to each other but this model wouldnt take it into account.

- create semantic embedings of the tags based on what topics they represent and vector of the documment woudl be mean of the tags it has
this aproach needs little more work, but solves the problem of related tags. for the sake of demonstration im using handwriten weights/vector embedings of few predeterminet tags though there are algoriths/ libraries that woudl make embeding out of any tag.
[math, coding, art, fantasy, gaming, hobby] this will be semantinc axis that I woudl use for each tag
exmple
    "NLA":   [0.90, 0.20, 0.00, 0.00, 0.00, 0.00] since its mainly about math but also we learn numerical methods for cooding


## Code and explanations

In [16]:
#importing libraries
import numpy as np

### norms 
few norms for testing
infinity norm is interesting as it will since its searching for maximum axis that woudl mean it would only take main topic of the document and igonore any other subtopic on semantic axis

In [17]:
def norm2 (a): return np.sqrt((a ** 2).sum(axis = -1))
def norm1 (a): return np.abs(a).sum(axis=-1)
def normifn (a): return np.max(np.abs(a), axis = -1)

### Clustering
to cluster entries efectifely I've created list of sets of intexes - 'clusters' in each set it woudd save indexes of points that clusters together. this wway we wont need to modify array containing points.
algorithm itself is simple we iterate over the list and find two closest clusters save their index(index in the cluster list) and distance if we find any pair that is closer we update and so on.
after we find two  closest we merge them rince and repeat untill we get down to the amount of clusters we want.

In [18]:
def clustering(array, numOfClusters, norm):
    clusters = [{idx} for idx in range(len(array))]

    while len(clusters) > numOfClusters:
        closestpair = None
        mindist = float("inf")

        for i in range(len(clusters)):
            for j in range(i + 1, len(clusters)):
                dist = cluster_distance(array, clusters[i], clusters[j], norm)
                if dist < mindist:
                    mindist = dist
                    closestpair = (i, j)

        i, j = closestpair
        merged = clusters[i] | clusters[j]

        clusters = [c for idx, c in enumerate(clusters) if idx not in (i, j)]
        clusters.append(merged)

    return clusters

### calculating distance betweein cluster
distance between two clusters is the distance between closest elements of these clusters
so we iterate over every possible pair and find shortest distance

In [19]:
def cluster_distance(points, clusterA, clusterB, norm):
    d = float("inf")
    for i in clusterA:
        for j in clusterB:
            dist = norm(points[i] - points[j])
            if dist < d:
                d = dist
    return d

### sample data and sample tag vectors

In [8]:
# semantic axes:
#[math, coding, art, fantasy, gaming, hobby]
tag_vectors = {
    "math":            np.array([1.00, 0.00, 0.00, 0.00, 0.00, 0.00]),
    "NLA":             np.array([0.90, 0.20, 0.00, 0.00, 0.00, 0.00]),
    "calculus":        np.array([0.85, 0.00, 0.00, 0.00, 0.00, 0.00]),
    "analysis":        np.array([0.80, 0.00, 0.00, 0.00, 0.00, 0.00]),

    "coding":          np.array([0.00, 1.00, 0.00, 0.00, 0.00, 0.00]),
    "python":          np.array([0.00, 0.90, 0.00, 0.00, 0.00, 0.00]),
    "cpp":             np.array([0.00, 0.80, 0.00, 0.00, 0.00, 0.00]),
    "algorithms":      np.array([0.10, 0.85, 0.00, 0.00, 0.00, 0.00]),
    "project":         np.array([0.00, 0.70, 0.00, 0.00, 0.00, 0.30]),  

    "art":             np.array([0.00, 0.00, 1.00, 0.00, 0.00, 0.00]),
    "design":          np.array([0.00, 0.00, 0.85, 0.00, 0.00, 0.00]),
    "drawing":         np.array([0.00, 0.00, 0.90, 0.00, 0.00, 0.30]),

    "fantasy":         np.array([0.00, 0.00, 0.00, 1.00, 0.00, 0.00]),
    "worldbuilding":   np.array([0.00, 0.00, 0.00, 0.90, 0.00, 0.30]),
    "lore":            np.array([0.00, 0.00, 0.00, 0.85, 0.00, 0.00]),
    "dnd":             np.array([0.00, 0.00, 0.00, 0.80, 0.30, 0.30]),  

    "gaming":          np.array([0.00, 0.00, 0.00, 0.00, 1.00, 0.00]),
    "game-dev":        np.array([0.00, 0.50, 0.00, 0.00, 0.90, 0.00]),
    "helldivers":      np.array([0.00, 0.00, 0.00, 0.00, 0.95, 0.00]),

    "hobby":           np.array([0.00, 0.00, 0.00, 0.00, 0.00, 1.00]),
    "journal":         np.array([0.00, 0.00, 0.00, 0.00, 0.00, 0.90]),
    "recipes":         np.array([0.00, 0.00, 0.00, 0.00, 0.00, 0.80])
}

In [9]:
#AI generated sample data
documents = {
    # Pure Math -----
    "la_notes":              ["math", "linear-algebra"],
    "calc_homework":         ["math", "calculus"],
    "analysis_summary":      ["math", "analysis"],
    "probability_notes":     ["math"],

    # ----- Pure Coding -----
    "python_basics":         ["coding", "python"],
    "cpp_optimizations":     ["coding", "cpp"],
    "algo_design":           ["coding", "algorithms"],
    "project_todo":          ["coding", "project"],

    # ----- Pure Art -----
    "portrait_sketch":       ["art", "drawing"],
    "ui_design_draft":       ["art", "design"],
    "color_theory":          ["art"],

    # ----- Pure Fantasy -----
    "dnd_session_1":         ["dnd", "worldbuilding"],
    "world_map":             ["fantasy", "worldbuilding"],
    "ancient_lore":          ["fantasy", "lore"],
    "magic_system":          ["fantasy"],

    # ----- Pure Gaming -----
    "game_review_hd2":       ["gaming", "helldivers"],
    "build_guide_slayer":    ["gaming"],
    "game_dev_plan":         ["gaming", "game-dev", "project"],

    # ----- Pure Hobby -----
    "daily_journal":         ["hobby", "journal"],
    "recipe_tacos":          ["hobby", "recipes"],
    "fitness_log":           ["hobby"],

    # ----- Math + Coding Mix -----
    "math_for_algos":        ["math", "algorithms"],
    "python_numerics":       ["coding", "math", "python"],
    "machine_learning_intro":["math", "coding", "algorithms", "python"],
    "project_summary":       ["coding", "math", "project"],

    # ----- Coding + Fantasy Mix -----
    "python_for_dnd":        ["coding", "dnd"],
    "campaign_tracker_tool": ["coding", "fantasy", "project"],
    "map_generator_code":    ["coding", "worldbuilding", "project"],
    "spell_calc_program":    ["coding", "fantasy", "math"],

    # ----- Art + Fantasy Mix -----
    "character_concepts":    ["art", "fantasy"],
    "world_map_art":         ["art", "worldbuilding"],
    "magic_creature_design": ["art", "fantasy", "design"],
    "ui_for_fantasy_game":   ["art", "gaming", "design"],

    # ----- Coding + Gaming Mix -----
    "ai_bot_for_games":      ["coding", "gaming", "algorithms"],
    "game_modding_notes":    ["gaming", "coding"],
    "game_dev_diary":        ["gaming", "project", "journal"],
    "procedural_generation": ["coding", "game-dev", "fantasy"],

    # ----- Weird Crossovers (stress tests) -----
    "math_of_color":         ["math", "art"],
    "fantasy_statistics":    ["math", "fantasy"],
    "cooking_for_dnd_party": ["hobby", "fantasy", "recipes"],
    "art_of_algorithms":     ["math", "coding", "art"],
    "lore_based_ai":         ["coding", "fantasy", "algorithms"],
    "game_study_notebook":   ["math", "gaming", "journal"],
    "dnd_budget_sheet":      ["math", "hobby", "dnd"],
    "fantasy_cooking":       ["fantasy", "recipes"],
}


In [15]:
def document_vector(tags):
    """Average vector of a set of tags."""
    vecs = [tag_vectors[tag] for tag in tags if tag in tag_vectors]
    if not vecs:
        return np.zeros(6)
    return np.mean(vecs, axis=0)
    
# converting docs to embedings 
doc_embeddings = {name: document_vector(tags) for name, tags in documents.items()}
array = np.vstack([vector  for vector in doc_embeddings.values()])


clusters = clustering(array,6, norm2)
for c in clusters:
    print({list(documents.keys())[i] for i in c}) 
    print("\n")

{'game_review_hd2', 'build_guide_slayer'}


{'la_notes', 'probability_notes', 'analysis_summary', 'calc_homework'}


{'magic_creature_design', 'character_concepts', 'world_map_art'}


{'project_todo', 'procedural_generation', 'campaign_tracker_tool', 'map_generator_code', 'project_summary', 'machine_learning_intro', 'fantasy_statistics', 'math_of_color', 'python_for_dnd', 'algo_design', 'python_numerics', 'spell_calc_program', 'ai_bot_for_games', 'art_of_algorithms', 'game_dev_plan', 'cpp_optimizations', 'math_for_algos', 'game_modding_notes', 'python_basics', 'lore_based_ai'}


{'recipe_tacos', 'dnd_session_1', 'fitness_log', 'daily_journal', 'game_study_notebook', 'cooking_for_dnd_party', 'dnd_budget_sheet', 'ancient_lore', 'fantasy_cooking', 'world_map', 'magic_system', 'game_dev_diary'}


{'color_theory', 'portrait_sketch', 'ui_for_fantasy_game', 'ui_design_draft'}


